# Compute the Daily temperature range (DTR)

In this example we compute the DTR as the monthly mean of the difference between the daily maximum temperature (TX) and minimum temperature (TN).

As first step, let's connect to the Ophidia Server

In [ ]:
import sys
from PyOphidia import cube,client
cube.Cube.setclient(read_env=True)

The JSON object associated to the workflow is shown in the cell below.

The task list includes the following tasks:

1. **Import**
 - the input NetCDF data sets located at ```src_path``` (set as a workflow input parameters) are imported into the Ophidia platform, with minimum and maximum temperature in °K (see http://ophidia.cmcc.it/documentation/users/operators/OPH_IMPORTNC.html)
 - the ```measure``` is set according to the workflow input parameters
 - data is arranged in order to operate on time series (as indicated by the ```imp_dim``` parameter)
 
 
2. **Daily DTR**
 - the *oph_intercube* operator (see http://ophidia.cmcc.it/documentation/users/operators/OPH_INTERCUBE.html) is used to execute an operation (*sub*) between the two imported datacubes 
 - the task has two **single** dependencies from the previous **Import** tasks since it uses one output from each of the parent tasks. In particular, the ```cube``` parameter of the oph_intercube operator will be set to the PID of the cube imported in the first Import task (*tasmax*), while the ```cube2``` parameter will be set to the PID of the cube imported in the second Import task (*tasmax*).
 

3. **Monthly DTR**
 - compute the monthly average of DTR
 - the *oph_reduce2* operator (see http://ophidia.cmcc.it/documentation/users/operators/OPH_REDUCE2.html) is used with ```operation=avg``` and ```concept_level=M```
 - **single** dependency from **Daily DTR** task


4. **Subset**
 - *oph_subset* operator (see http://ophidia.cmcc.it/documentation/users/operators/OPH_SUBSET.html) to extract the first month (Jan 2096) providing the time range in human-readable form: ```subset_filter=2096-01```, ```subset_dims=time```, ```subset_type=coord```
 - **single** dependency from **Monthly DTR** task

5. **Export**
 - *oph_exportnc2* operator (see http://ophidia.cmcc.it/documentation/users/operators/OPH_EXPORTNC2.html) to export the DTR index related to January 2096 to a NetCDF file
 - **single** dependency from **task 4**
 
 
<img src="../imgs/DTR.svg" alt="Summer_Days">

Note that the two *oph_importnc* tasks (for *tasmax* and *tasmin*) are independent from each other and can be executed concurrently by the workflow management system. Each task defines two variables in order to define at submission time both the NetCDF path and the variable name.

In [ ]:
dtr_workflow = """{
        "name": "DTR",
        "author": "CMCC",
        "abstract": "Workflow version of the Daily temperature range (DTR) index",
        "exec_mode": "sync",
        "ncores": "2",
        "cwd": "/",
        "tasks":
        [
                {
                        "name": "Import tasmin",
                        "operator": "oph_importnc",
                        "arguments":
                        [
                                "src_path=$1",
                                "measure=$2",
                                "import_metadata=yes",
                                "imp_dim=time",
                                "imp_concept_level=d",
                                "vocabulary=CF",
                                "hierarchy=oph_base|oph_base|oph_time",
                                "description=Min Temp"
                        ]
                },
                {
                        "name": "Import tasmax",
                        "operator": "oph_importnc",
                        "arguments":
                        [
                                "src_path=$3",
                                "measure=$4",
                                "import_metadata=yes",
                                "imp_dim=time",
                                "imp_concept_level=d",
                                "vocabulary=CF",
                                "hierarchy=oph_base|oph_base|oph_time",
                                "description=Max Temp"
                        ]
                },
                {
                        "name": "Daily DTR",
                        "operator": "oph_intercube",
                        "arguments":
                        [
                                "operation=sub",
                                "description=Daily DTR"
                        ],
                        "dependencies": [
                                { "task": "Import tasmin", "type": "single", "argument":"cube2" },
                                { "task": "Import tasmax", "type": "single", "argument":"cube" }
                        ]
                },
                {
                        "name": "Monthly DTR",
                        "operator": "oph_reduce2",
                        "arguments":
                        [
                                "operation=avg",
                                "dim=time",
                                "concept_level=M",
                                "description=Monthly DTR"
                        ],
                        "dependencies": [
                                { "task": "Daily DTR", "type": "single" }
                        ]
                },
                {
                        "name": "First month",
                        "operator": "oph_subset",
                        "arguments":
                        [
                                "subset_filter=2096-01",
                                "subset_dims=time",
                                "description=First year",
                                "subset_type=coord"
                        ],
                        "dependencies": [
                                { "task": "Monthly DTR", "type": "single" }
                        ]
                },
                {
                        "name": "Export",
                        "operator": "oph_exportnc2",
                        "arguments": [
                            "output_name=DTR_2096-01",
                            "output_path=/home/ophidia/notebooks/"
                        ],
                        "dependencies": [
                            { "task": "First month", "type": "single"}
                        ]
                }
        ]
            
}"""

Once the workflow is defined, it can be executed very easily on different dataset by simply specifying the proper path and variable name for the minimum and maximum temperature.

Let's define the workflow input arguments for the example

In [ ]:
path="/home/ophidia/notebooks/"
file1="tasmin_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc"
variable1="tasmin"
file2="tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc"
variable2="tasmax"

Submit the workflow

In [ ]:
cube.Cube.client.wsubmit(dtr_workflow, path+file1, variable1, path+file2, variable2)

Check for DTR_2096-01.nc

In [ ]:
import glob
glob.glob('/home/ophidia/notebooks/DTR*.nc')

We can plot a map for year 2096 by considering the PID associated to the 'First year' datacube

In [ ]:
cube.Cube.list(level=2)

In [ ]:
# Get PID of 'First year'
firstyear = cube.Cube(pid='http://127.0.0.1/ophidia/.../...')

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fig = plt.figure(figsize=(15, 6), dpi=100)

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)

#Draw coastline and gridlines
ax.coastlines()

gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
gl.xlabels_top = False
gl.ylabels_right = False

data = firstyear.export_array(show_time='yes')
lat = data['dimension'][0]['values'][ : ]
lon = data['dimension'][1]['values'][ : ]
var = data['measure'][0]['values'][ : ]
var = np.reshape(var, (len(lat), len(lon)))

#Wraparound points in longitude
var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
x, y = np.meshgrid(lon_cyclic,lat)

#Define color levels for color bar
levStep = (np.nanmax(var)-np.nanmin(var))/20
clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

#Set filled contour plot
cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.Oranges)
plt.colorbar(cnplot,ax=ax)

ax.set_aspect('auto', adjustable=None)

plt.title('Frost Days (year 2096)')
plt.show()

Before running the other examples, empty the workspace

In [ ]:
cube.Cube.deletecontainer(container=file1,force='yes')
cube.Cube.deletecontainer(container=file2,force='yes')

In [ ]:
cube.Cube.list(level=2)